In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 15.9 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import shutil # to save it locally

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= 'CLIENT_ID',
                                                           client_secret= 'CLIENT_SECRET'),
                    requests_timeout=30)

In [ ]:
def get_playlist_tracks(playlist_id, n_tracks):
    data = pd.DataFrame()
    #spotify API sets a default limit of 100 tracks, to get all we need to invoke offset
    for i in range(0,n_tracks,100):
        results = sp.playlist_tracks(playlist_id,limit=100, offset=i)
        #shape json into dataframe
        track_ids = [track.get('track').get('id') for track in results.get('items')]
        df = pd.DataFrame()
        df['id'] = track_ids
        df['track'] = [track.get('track').get('name') for track in results.get('items')]
        df['artist_id'] = [track.get('track').get('artists')[0].get('id') for track in results.get('items')]
        df['artist'] = [track.get('track').get('artists')[0].get('name') for track in results.get('items')]
        df['album_id'] = [track.get('track').get('album').get('id') for track in results.get('items')]
        df['album'] = [track.get('track').get('album').get('name') for track in results.get('items')]
        df['release_date'] = [track.get('track').get('album').get('release_date') for track in results.get('items')]
        df['link'] = [track.get('track').get('href') for track in results.get('items')]
        #get audio features, merge with df
        track_features = sp.audio_features(track_ids)
        df_features = pd.DataFrame.from_dict(track_features)
        df = df.merge(df_features, on='id', how='left')
        #merge with data
        data = pd.concat([data,df])
    return data

In [ ]:
df_total500 = get_playlist_tracks('0ZZhm1GsunQmAd0VG3Nop0', 500)

In [ ]:
df_total500.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,0.668,0.727,...,0.000019,0.1000,0.939,139.991,audio_features,spotify:track:3pCt2wRdBDa2kCisIdHWgF,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,https://api.spotify.com/v1/audio-analysis/3pCt...,344571,4
1,6Hii26x3qDErVitnGW8QtO,Runtuh,66NmU5epI0ONGmdCRbLpmW,Feby Putri,5boeKAXsTkdFlm6OnkQxQW,Runtuh,2021-10-01,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,0.593,0.287,...,0.000136,0.1190,0.191,127.783,audio_features,spotify:track:6Hii26x3qDErVitnGW8QtO,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,https://api.spotify.com/v1/audio-analysis/6Hii...,223974,4
2,2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,2iDVt6mFbtbDEZG5ax0dTi,Tulus,3R4IAF9ApqYeUQrv1ddyoR,Manusia,2022-03-03,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,0.643,0.436,...,0.088900,0.1150,0.755,71.969,audio_features,spotify:track:2hHeGD57S0BcopfVcmehdl,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,https://api.spotify.com/v1/audio-analysis/2hHe...,242000,4
3,630DpnzdfjdVqv2yLfPbAX,Bertaut,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,0.470,0.131,...,0.002450,0.1090,0.115,118.836,audio_features,spotify:track:630DpnzdfjdVqv2yLfPbAX,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,https://api.spotify.com/v1/audio-analysis/630D...,315961,3
4,4GfK1qOF3uBWidbPlTCQRL,Monokrom,2iDVt6mFbtbDEZG5ax0dTi,Tulus,4szhn3xPmOJklFAcqNvTnQ,Monokrom,2016-08-03,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,0.534,0.462,...,0.000006,0.0974,0.519,88.046,audio_features,spotify:track:4GfK1qOF3uBWidbPlTCQRL,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,https://api.spotify.com/v1/audio-analysis/4GfK...,214567,4


In [ ]:
df_total500.shape

(500, 25)

In [ ]:
# Here with me
df_total600 = get_playlist_tracks('1NIjPA1o5vAAWzLhBv35w5', 100)

In [ ]:
df_total600.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5icOoE6VgqFKohjWWNp0Ac,Here With Me,64KEffDW9EtZ1y2vBYgq8T,Marshmello,6NHS3hV16MZyfcp0nSHdrd,Here With Me,2019-03-08,https://api.spotify.com/v1/tracks/5icOoE6VgqFK...,0.791,0.565,...,0.000000,0.156,0.1810,99.961,audio_features,spotify:track:5icOoE6VgqFKohjWWNp0Ac,https://api.spotify.com/v1/tracks/5icOoE6VgqFK...,https://api.spotify.com/v1/audio-analysis/5icO...,156347,4
1,7zPiwqd1uxBkiHuk2OoBWJ,Cinta Sejati,30qVSJGhPhrZLKG0H9DMA9,Bunga Citra Lestari,3cMuELN6TQ1pF5fkOcdhpv,Cinta Sejati,2012,https://api.spotify.com/v1/tracks/7zPiwqd1uxBk...,0.220,0.402,...,0.000194,0.147,0.2240,177.396,audio_features,spotify:track:7zPiwqd1uxBkiHuk2OoBWJ,https://api.spotify.com/v1/tracks/7zPiwqd1uxBk...,https://api.spotify.com/v1/audio-analysis/7zPi...,300933,4
2,4l0Mvzj72xxOpRrp6h8nHi,Lose You To Love Me,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,3YPFaTR7WMi1Hd4NVKdCJx,Rare,2020-01-10,https://api.spotify.com/v1/tracks/4l0Mvzj72xxO...,0.488,0.343,...,0.000000,0.210,0.0978,102.819,audio_features,spotify:track:4l0Mvzj72xxOpRrp6h8nHi,https://api.spotify.com/v1/tracks/4l0Mvzj72xxO...,https://api.spotify.com/v1/audio-analysis/4l0M...,206459,4
3,3gBFAvpAVSNohs3ESRMYHi,Heaven,05CRzFTp7TouOXPuH6Tapu,Isyana Sarasvati,3GJjPtV0iPKM3fLPvqdURX,Heaven,2018-02-14,https://api.spotify.com/v1/tracks/3gBFAvpAVSNo...,0.540,0.285,...,0.000000,0.152,0.4340,87.789,audio_features,spotify:track:3gBFAvpAVSNohs3ESRMYHi,https://api.spotify.com/v1/tracks/3gBFAvpAVSNo...,https://api.spotify.com/v1/audio-analysis/3gBF...,218507,4
4,5Z3GHaZ6ec9bsiI5BenrbY,Young Dumb & Broke,6LuN9FCkKOj5PcnpouEgny,Khalid,6kf46HbnYCZzP6rjvQHYzg,American Teen,2017-04-27,https://api.spotify.com/v1/tracks/5Z3GHaZ6ec9b...,0.799,0.539,...,0.000017,0.165,0.3940,136.948,audio_features,spotify:track:5Z3GHaZ6ec9bsiI5BenrbY,https://api.spotify.com/v1/tracks/5Z3GHaZ6ec9b...,https://api.spotify.com/v1/audio-analysis/5Z3G...,202547,4


In [ ]:
df_total600.shape

(100, 25)

In [ ]:
#df_total700 = get_playlist_tracks('45MRo66QGkNhHxyBJtA4Rq', 100)

In [ ]:
df_total800 = get_playlist_tracks('6EenOW3gjYDLvcLF8GqyPE', 300)

In [ ]:
df_total800.shape

(300, 25)

In [ ]:
df_total800.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4EivmOT13NMpNSfTKn9p4s,Like That (feat. Gucci Mane),5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,1MmVkhiwTH0BkNOU3nw5d3,Hot Pink,2019-11-07,https://api.spotify.com/v1/tracks/4EivmOT13NMp...,0.879,0.498,...,0.000000,0.0967,0.405,106.044,audio_features,spotify:track:4EivmOT13NMpNSfTKn9p4s,https://api.spotify.com/v1/tracks/4EivmOT13NMp...,https://api.spotify.com/v1/audio-analysis/4Eiv...,163173,4
1,44ONERBHALxftQNWq6dmyd,Side To Side,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,1QRP5lutJodPixU2EWfnD7,Dangerous Woman,2016-04-01,https://api.spotify.com/v1/tracks/44ONERBHALxf...,0.650,0.736,...,0.000000,0.2350,0.612,159.174,audio_features,spotify:track:44ONERBHALxftQNWq6dmyd,https://api.spotify.com/v1/tracks/44ONERBHALxf...,https://api.spotify.com/v1/audio-analysis/44ON...,226160,4
2,1oPhiw7KmW3JK0gjuSF4YH,Menyimpan Rasa,5nWpcynsaMBFgtGmJODhdi,Devano,0E8TQKFgUP04XaMYK1ls1m,Menyimpan Rasa,2018-03-20,https://api.spotify.com/v1/tracks/1oPhiw7KmW3J...,0.603,0.469,...,0.000002,0.1760,0.681,74.998,audio_features,spotify:track:1oPhiw7KmW3JK0gjuSF4YH,https://api.spotify.com/v1/tracks/1oPhiw7KmW3J...,https://api.spotify.com/v1/audio-analysis/1oPh...,210197,4
3,2YBKZDLTHfoqe7IUEe2z6n,Teman Tapi Menikah,0FYCcpBLjjMMnse9jIRbu3,Dengarkan Dia,61pxeWGW8w4LHpLorcVzL7,Teman Tapi Menikah,2017-02-02,https://api.spotify.com/v1/tracks/2YBKZDLTHfoq...,0.702,0.307,...,0.000010,0.0844,0.410,111.987,audio_features,spotify:track:2YBKZDLTHfoqe7IUEe2z6n,https://api.spotify.com/v1/tracks/2YBKZDLTHfoq...,https://api.spotify.com/v1/audio-analysis/2YBK...,222250,4
4,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,1uyf3l2d4XYwiEqAb7t7fX,Doo-Wops & Hooligans,2010-10-05,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,0.635,0.841,...,0.000000,0.0622,0.424,109.021,audio_features,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,https://api.spotify.com/v1/audio-analysis/7BqB...,220734,4


In [ ]:
df_total1500 = get_playlist_tracks('5WIZWmYoXnsotpHDcyLdKp', 500)

In [ ]:
df_total1500.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,7JFFMicliyVArHQgwEqVBS,Alexandra,1DjZI46mVZZZYmmmygRnTw,Reality Club,6xKDnJaJSGOyu2nUmOQlJC,What Do You Really Know?,2019-08-30,https://api.spotify.com/v1/tracks/7JFFMicliyVA...,0.382,0.355,...,0.000003,0.2480,0.146,138.419,audio_features,spotify:track:7JFFMicliyVArHQgwEqVBS,https://api.spotify.com/v1/tracks/7JFFMicliyVA...,https://api.spotify.com/v1/audio-analysis/7JFF...,238543,4
1,7uP1XTPVEI40AZGVKNzITm,Sang Penggoda,6reQVU3OFzQdP0NqZCwrCz,Tata Janeeta,2LtXJXiTmmd2mGbVi7PzLH,Sang Penggoda,2018-04-13,https://api.spotify.com/v1/tracks/7uP1XTPVEI40...,0.354,0.403,...,0.000000,0.1240,0.214,118.100,audio_features,spotify:track:7uP1XTPVEI40AZGVKNzITm,https://api.spotify.com/v1/tracks/7uP1XTPVEI40...,https://api.spotify.com/v1/audio-analysis/7uP1...,271804,4
2,7ju97lgwC2rKQ6wwsf9no9,Rain On Me (with Ariana Grande),1HY2Jd0NmPuamShAr6KMms,Lady Gaga,05c49JgPmL4Uz2ZeqRx5SP,Chromatica,2020-05-29,https://api.spotify.com/v1/tracks/7ju97lgwC2rK...,0.672,0.855,...,0.000000,0.3230,0.646,123.056,audio_features,spotify:track:7ju97lgwC2rKQ6wwsf9no9,https://api.spotify.com/v1/tracks/7ju97lgwC2rK...,https://api.spotify.com/v1/audio-analysis/7ju9...,182200,4
3,1AZjCZSGEjlxUufFQwSszF,Wild Flower (with youjeen),2auC28zjQyVTsiZKNgPRGs,RM,2wGinO7YWLHN2sULIr4a7v,Indigo,2022-12-02,https://api.spotify.com/v1/tracks/1AZjCZSGEjlx...,0.486,0.766,...,0.000000,0.1230,0.417,154.992,audio_features,spotify:track:1AZjCZSGEjlxUufFQwSszF,https://api.spotify.com/v1/tracks/1AZjCZSGEjlx...,https://api.spotify.com/v1/audio-analysis/1AZj...,273500,4
4,20R4HfKloPKgXDqU7UKk3x,Therefore I Am,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,0JGOiO34nwfUdDrD612dOp,Happier Than Ever,2021-07-30,https://api.spotify.com/v1/tracks/20R4HfKloPKg...,0.879,0.365,...,0.135000,0.0583,0.663,94.016,audio_features,spotify:track:20R4HfKloPKgXDqU7UKk3x,https://api.spotify.com/v1/tracks/20R4HfKloPKg...,https://api.spotify.com/v1/audio-analysis/20R4...,173539,4


In [ ]:
df_total1500.shape

(500, 25)

In [ ]:
# Love Me Now
df_total2000 = get_playlist_tracks('72duosF7lkFT9p80nciOyU', 500)

In [ ]:
df_total2000.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,6nxQdXa1uAL0rY72wPZu89,Love Me Now,5y2Xq6xcjJb2jVM54GHK3t,John Legend,7xMjYDrgPLp1ReFGAOyS1O,DARKNESS AND LIGHT,2016-12-02,https://api.spotify.com/v1/tracks/6nxQdXa1uAL0...,0.495,0.762,...,0.0000,0.0763,0.708,123.893,audio_features,spotify:track:6nxQdXa1uAL0rY72wPZu89,https://api.spotify.com/v1/tracks/6nxQdXa1uAL0...,https://api.spotify.com/v1/audio-analysis/6nxQ...,210293,4
1,6NEoeBLQbOMw92qMeLfI40,WHISTLE,41MozSoPIsD1dJM0CLPjZF,BLACKPINK,0FOOodYRlj7gzh7q7IjmNZ,SQUARE ONE,2016-08-08,https://api.spotify.com/v1/tracks/6NEoeBLQbOMw...,0.818,0.723,...,0.0000,0.0866,0.607,102.945,audio_features,spotify:track:6NEoeBLQbOMw92qMeLfI40,https://api.spotify.com/v1/tracks/6NEoeBLQbOMw...,https://api.spotify.com/v1/audio-analysis/6NEo...,211632,4
2,5SjfjoYaRJ5jycgqwV0ow0,Scars,6Ge3YISmUWPCUcSVx8VGor,Keenan Te,5ZFNVSacvOj5XKRkqv39tY,Scars,2023-02-17,https://api.spotify.com/v1/tracks/5SjfjoYaRJ5j...,0.571,0.496,...,0.0000,0.1070,0.383,144.925,audio_features,spotify:track:5SjfjoYaRJ5jycgqwV0ow0,https://api.spotify.com/v1/tracks/5SjfjoYaRJ5j...,https://api.spotify.com/v1/audio-analysis/5Sjf...,192000,4
3,0b9oOr2ZgvyQu88wzixux9,This Is America,73sIBHcqh3Z3NyqHKZ7FOL,Childish Gambino,7arx9qPJexCsDz67El4qvk,This Is America,2018-05-06,https://api.spotify.com/v1/tracks/0b9oOr2ZgvyQ...,0.854,0.463,...,0.0000,0.3540,0.549,120.024,audio_features,spotify:track:0b9oOr2ZgvyQu88wzixux9,https://api.spotify.com/v1/tracks/0b9oOr2ZgvyQ...,https://api.spotify.com/v1/audio-analysis/0b9o...,225773,4
4,7eOkHIcaoUhngxxypM5Ajr,Look At Her Now,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,3YPFaTR7WMi1Hd4NVKdCJx,Rare,2020-01-10,https://api.spotify.com/v1/tracks/7eOkHIcaoUhn...,0.753,0.564,...,0.0273,0.3590,0.550,77.342,audio_features,spotify:track:7eOkHIcaoUhngxxypM5Ajr,https://api.spotify.com/v1/tracks/7eOkHIcaoUhn...,https://api.spotify.com/v1/audio-analysis/7eOk...,162596,5


In [ ]:
df_total2000.shape

(500, 25)

In [ ]:
# Dumhdurum
df_total2500 = get_playlist_tracks('5Gf25E31ZCmdL96rcU8WEn', 500)

In [ ]:
df_total2500.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,55kuHkT2CWhWKF6CoRDnJq,Dumhdurum,2uWcrwgWmZcQc3IPBs3tfU,Apink,49NKBZDb92AQlMFypkZRTF,LOOK,2020-04-13,https://api.spotify.com/v1/tracks/55kuHkT2CWhW...,0.653,0.923,...,0.0,0.137,0.6920,114.902,audio_features,spotify:track:55kuHkT2CWhWKF6CoRDnJq,https://api.spotify.com/v1/tracks/55kuHkT2CWhW...,https://api.spotify.com/v1/audio-analysis/55ku...,208053,4
1,6FtLXwyMr1t1C7xTUBL9uY,MOVE,5KOTrsUUc0VUVH3mR5StNC,SIXC (6 crazy),0dK8TElRE9K5AyO31YptuE,PRODUCE X 101 - 31 Boys 5 Concepts,2019-07-05,https://api.spotify.com/v1/tracks/6FtLXwyMr1t1...,0.733,0.782,...,0.0,0.403,0.8180,106.892,audio_features,spotify:track:6FtLXwyMr1t1C7xTUBL9uY,https://api.spotify.com/v1/tracks/6FtLXwyMr1t1...,https://api.spotify.com/v1/audio-analysis/6FtL...,185400,4
2,14pjnaIqkpReO5D0tHofAS,Daisy,1wKpMkucynaTfG8lyPprYV,PENTAGON,1ASYbBYBwV6Rcfc2ycqmlK,WE:TH,2020-10-12,https://api.spotify.com/v1/tracks/14pjnaIqkpRe...,0.563,0.869,...,0.0,0.138,0.4800,85.016,audio_features,spotify:track:14pjnaIqkpReO5D0tHofAS,https://api.spotify.com/v1/tracks/14pjnaIqkpRe...,https://api.spotify.com/v1/audio-analysis/14pj...,189000,4
3,2MdoXFLKNUnTdgWZisagEp,You Are My Everything (feat. Red),4rUYk0fV0Z4pOtwVbEAyK9,Glenn Fredly,5qPDSS9JkOf6As1DqdhKpF,OST. Cinta Silver,2005-06-24,https://api.spotify.com/v1/tracks/2MdoXFLKNUnT...,0.590,0.689,...,0.0,0.190,0.5460,104.985,audio_features,spotify:track:2MdoXFLKNUnTdgWZisagEp,https://api.spotify.com/v1/tracks/2MdoXFLKNUnT...,https://api.spotify.com/v1/audio-analysis/2Mdo...,306227,4
4,7KokYm8cMIXCsGVmUvKtqf,Karma,06HL4z0CvFAxyc27GXpf02,Taylor Swift,151w1FgRZfnKZA9FEcg9Z3,Midnights,2022-10-21,https://api.spotify.com/v1/tracks/7KokYm8cMIXC...,0.642,0.620,...,0.0,0.483,0.0969,90.020,audio_features,spotify:track:7KokYm8cMIXCsGVmUvKtqf,https://api.spotify.com/v1/tracks/7KokYm8cMIXC...,https://api.spotify.com/v1/audio-analysis/7Kok...,204852,4


In [ ]:
df_total2500.shape

(500, 25)

In [ ]:
# Jadi Milikku
df_total3000 = get_playlist_tracks('3vfh9gJTpeYhtq4WRDnvwG', 500)

In [ ]:
df_total3000.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,29euE2RV1DoN6aJXm3wmcS,Jadi Milikku,0JvUFDnuWPbuyRa6ptVkLk,Marcell,4lbLWbnI1sbBsPK5lYNprT,Jadi Milikku,2017-07-06,https://api.spotify.com/v1/tracks/29euE2RV1DoN...,0.586,0.404,...,0.000000,0.1010,0.391,115.913,audio_features,spotify:track:29euE2RV1DoN6aJXm3wmcS,https://api.spotify.com/v1/tracks/29euE2RV1DoN...,https://api.spotify.com/v1/audio-analysis/29eu...,251897,4
1,3D110t9ctcoOoylXsc3iR8,Thick And Thin,49tQo2QULno7gxHutgccqF,LANY,6SWmGozzQDUaczHXMuE8Za,Malibu Nights,2018-10-05,https://api.spotify.com/v1/tracks/3D110t9ctcoO...,0.697,0.551,...,0.000000,0.0762,0.339,99.996,audio_features,spotify:track:3D110t9ctcoOoylXsc3iR8,https://api.spotify.com/v1/tracks/3D110t9ctcoO...,https://api.spotify.com/v1/audio-analysis/3D11...,212002,4
2,1WjGvMbHE23jEipydV80Mu,FAKE LOVE - Rocking Vibe Mix,3Nrfpe0tUJi4K4DXYWgMUX,BTS,43wFM1HquliY3iwKWzPN4y,Love Yourself 結 'Answer',2018-08-24,https://api.spotify.com/v1/tracks/1WjGvMbHE23j...,0.466,0.804,...,0.000000,0.4320,0.352,154.901,audio_features,spotify:track:1WjGvMbHE23jEipydV80Mu,https://api.spotify.com/v1/tracks/1WjGvMbHE23j...,https://api.spotify.com/v1/audio-analysis/1WjG...,238494,4
3,7kt9e9LFSpN1zQtYEl19o1,this is me trying,06HL4z0CvFAxyc27GXpf02,Taylor Swift,2fenSS68JI1h4Fo296JfGr,folklore,2020-07-24,https://api.spotify.com/v1/tracks/7kt9e9LFSpN1...,0.515,0.476,...,0.000047,0.1710,0.412,135.886,audio_features,spotify:track:7kt9e9LFSpN1zQtYEl19o1,https://api.spotify.com/v1/tracks/7kt9e9LFSpN1...,https://api.spotify.com/v1/audio-analysis/7kt9...,195097,4
4,5oO3drDxtziYU2H1X23ZIp,Love On The Brain,5pKCCKE2ajJHZ9KAiaK11H,Rihanna,4UlGauD7ROb3YbVOFMgW5u,ANTI (Deluxe),2016-01-28,https://api.spotify.com/v1/tracks/5oO3drDxtziY...,0.509,0.637,...,0.000011,0.0789,0.378,172.006,audio_features,spotify:track:5oO3drDxtziYU2H1X23ZIp,https://api.spotify.com/v1/tracks/5oO3drDxtziY...,https://api.spotify.com/v1/audio-analysis/5oO3...,224000,3


In [ ]:
df_total3000.shape

(500, 25)

In [ ]:
# Walking The Wire
df_total3500 = get_playlist_tracks('4x6z71qoyCA0VhLYGUyJcI', 500)

In [ ]:
df_total3500.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,1y3bE5i57eUm3hfLAc7h08,Walking The Wire,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,33pt9HBdGlAbRGBHQgsZsU,Evolve,2017-06-23,https://api.spotify.com/v1/tracks/1y3bE5i57eUm...,0.540,0.761,...,0.000000,0.0941,0.147,100.008,audio_features,spotify:track:1y3bE5i57eUm3hfLAc7h08,https://api.spotify.com/v1/tracks/1y3bE5i57eUm...,https://api.spotify.com/v1/audio-analysis/1y3b...,232907,4
1,0heY4hxgRs6s1wrMtrS3YJ,Tak Pernah Padam,4sZCNSc9bpEa6bjM8Z3NOu,Sandhy Sondoro,3kOVbZ8Ccl83UB9cGTvvi5,Find The Way,2011-12-16,https://api.spotify.com/v1/tracks/0heY4hxgRs6s...,0.667,0.456,...,0.000000,0.1700,0.551,74.987,audio_features,spotify:track:0heY4hxgRs6s1wrMtrS3YJ,https://api.spotify.com/v1/tracks/0heY4hxgRs6s...,https://api.spotify.com/v1/audio-analysis/0heY...,241107,4
2,3p287E4Y1LOqbSICfbv1hk,The Days We Loved,3l0MJ9LCiBlMyOMaz485Q5,Baek Ji Young,3giN7KiHhfRJb2L8LGToHu,"The World of the Married, Pt. 6 (Original Tele...",2020-05-01,https://api.spotify.com/v1/tracks/3p287E4Y1LOq...,0.367,0.627,...,0.000000,0.1050,0.380,137.013,audio_features,spotify:track:3p287E4Y1LOqbSICfbv1hk,https://api.spotify.com/v1/tracks/3p287E4Y1LOq...,https://api.spotify.com/v1/audio-analysis/3p28...,241053,4
3,38iufYNK4hUzS649Oe2kZy,Burn It (feat. MAX),5RmQ8k4l3HZ8JoPb4mNsML,Agust D,1Pp8t7yn2E3rz3R7ZqPn1O,D-2,2020-05-22,https://api.spotify.com/v1/tracks/38iufYNK4hUz...,0.658,0.776,...,0.000002,0.2720,0.336,114.044,audio_features,spotify:track:38iufYNK4hUzS649Oe2kZy,https://api.spotify.com/v1/tracks/38iufYNK4hUz...,https://api.spotify.com/v1/audio-analysis/38iu...,192107,4
4,2aNkofLzdkgsCpC0AzEJB4,What We Remember,3ahVFszO9ucf8JQLCNQeYA,Anggun,3m7KxVv2s1VhrfJh6cinck,8,2017-12-08,https://api.spotify.com/v1/tracks/2aNkofLzdkgs...,0.800,0.677,...,0.000428,0.1760,0.387,114.041,audio_features,spotify:track:2aNkofLzdkgsCpC0AzEJB4,https://api.spotify.com/v1/tracks/2aNkofLzdkgs...,https://api.spotify.com/v1/audio-analysis/2aNk...,166294,4


In [ ]:
df_total3500.shape

(502, 25)

In [ ]:
# All is Found
df_total4000 = get_playlist_tracks('2YNbrDfLUrbYgDj2ARjfCh', 500)

In [ ]:
df_total4000.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,61HVbcNeRACZpyvHrc3AnD,All Is Found,3MyB0MyKec3Jq0AH2llk7v,Evan Rachel Wood,4M07HWIlZr7zoXoxDHR5mz,Frozen 2 (Original Motion Picture Soundtrack/D...,2019-11-15,https://api.spotify.com/v1/tracks/61HVbcNeRACZ...,0.228,0.204,...,0.000011,0.092,0.195,83.372,audio_features,spotify:track:61HVbcNeRACZpyvHrc3AnD,https://api.spotify.com/v1/tracks/61HVbcNeRACZ...,https://api.spotify.com/v1/audio-analysis/61HV...,125439,4
1,6smhpAoDKenJyDztNHf0HH,Thinking About You,2XnBwblw31dfGnspMIwgWz,Axwell /\ Ingrosso,6Ei5WWek37m1x9AjpSXTcJ,More Than You Know,2017-07-28,https://api.spotify.com/v1/tracks/6smhpAoDKenJ...,0.678,0.842,...,0.000000,0.202,0.493,122.016,audio_features,spotify:track:6smhpAoDKenJyDztNHf0HH,https://api.spotify.com/v1/tracks/6smhpAoDKenJ...,https://api.spotify.com/v1/audio-analysis/6smh...,204587,4
2,28foEMQSiO6arnHRX8HdHj,Like always,6C4irZ60X8G7UimMRQiCcg,X1,3zgalFg8VBaUSHKeLBVGun,QUANTUM LEAP,2019-08-27,https://api.spotify.com/v1/tracks/28foEMQSiO6a...,0.782,0.879,...,0.000000,0.047,0.822,119.978,audio_features,spotify:track:28foEMQSiO6arnHRX8HdHj,https://api.spotify.com/v1/tracks/28foEMQSiO6a...,https://api.spotify.com/v1/audio-analysis/28fo...,204893,4
3,7gzdvN8XbAXwYFOOUj7CxM,hormat kepada angin,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,2H7YgnGyrAi2AZsskG6Ows,kalah bertaruh,2021-05-26,https://api.spotify.com/v1/tracks/7gzdvN8XbAXw...,0.480,0.354,...,0.031600,0.137,0.221,138.071,audio_features,spotify:track:7gzdvN8XbAXwYFOOUj7CxM,https://api.spotify.com/v1/tracks/7gzdvN8XbAXw...,https://api.spotify.com/v1/audio-analysis/7gzd...,355627,4
4,4mU5iXHeLgbR94siF7p1sY,Six Feet Under,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,2ODvWsOgouMbaA5xf0RkJe,Starboy,2016-11-25,https://api.spotify.com/v1/tracks/4mU5iXHeLgbR...,0.772,0.504,...,0.000000,0.125,0.244,140.035,audio_features,spotify:track:4mU5iXHeLgbR94siF7p1sY,https://api.spotify.com/v1/tracks/4mU5iXHeLgbR...,https://api.spotify.com/v1/audio-analysis/4mU5...,237573,4


In [ ]:
df_total4000.shape

(500, 25)

In [ ]:
# Sing Me To Sleep
df_total4477 = get_playlist_tracks('6CrCEztziK818weaqUNdqp', 477)

In [ ]:
df_total4477.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2BcwzjPvVSkek2ZLdh7C9J,Sing Me to Sleep - Marshmello Remix,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,4wQNBX0Mp50Q2JFSEDW5G1,Sing Me to Sleep,2016-06-03,https://api.spotify.com/v1/tracks/2BcwzjPvVSke...,0.620,0.893,...,0.000304,0.1590,0.160,101.089,audio_features,spotify:track:2BcwzjPvVSkek2ZLdh7C9J,https://api.spotify.com/v1/tracks/2BcwzjPvVSke...,https://api.spotify.com/v1/audio-analysis/2Bcw...,192480,4
1,3w9mof4ZxG3z9huYwHFZ1i,That's It (feat. Gucci Mane & 2 Chainz),64M6ah0SkkRsnPGtGiRAbb,Bebe Rexha,6t5D6LEgHxqUVOxJItkzfb,All Your Fault: Pt. 2,2017-08-11,https://api.spotify.com/v1/tracks/3w9mof4ZxG3z...,0.783,0.361,...,0.007620,0.0861,0.413,75.000,audio_features,spotify:track:3w9mof4ZxG3z9huYwHFZ1i,https://api.spotify.com/v1/tracks/3w9mof4ZxG3z...,https://api.spotify.com/v1/audio-analysis/3w9m...,207090,4
2,6FT9FZccFD6nE8dMNslz2n,Walk On Water,0RqtSIYZmd4fiBKVFqyIqD,Thirty Seconds To Mars,0XcHdI2ZyNADjfvo5Ubs39,AMERICA,2018-04-06,https://api.spotify.com/v1/tracks/6FT9FZccFD6n...,0.495,0.787,...,0.000000,0.1670,0.388,140.003,audio_features,spotify:track:6FT9FZccFD6nE8dMNslz2n,https://api.spotify.com/v1/tracks/6FT9FZccFD6n...,https://api.spotify.com/v1/audio-analysis/6FT9...,185467,4
3,46coHhbK0YFKlnh3qYtVBB,I Did It,2fS9sWFJcWN8wVhYbCfdC7,Teza Sumendra,0ULXt1LoNwUlyWVbAjfQMU,I Did It,2018-03-16,https://api.spotify.com/v1/tracks/46coHhbK0YFK...,0.726,0.681,...,0.000047,0.1470,0.635,116.954,audio_features,spotify:track:46coHhbK0YFKlnh3qYtVBB,https://api.spotify.com/v1/tracks/46coHhbK0YFK...,https://api.spotify.com/v1/audio-analysis/46co...,336466,4
4,0My8NPmENHrN5W7OfgZnZJ,The One,4BxCuXFJrSWGi1KHcVqaU4,Kodaline,3IJ5RDbQeiWIf7Fo601JwE,Coming Up for Air (Expanded Edition),2015-02-09,https://api.spotify.com/v1/tracks/0My8NPmENHrN...,0.430,0.587,...,0.000088,0.1580,0.332,156.006,audio_features,spotify:track:0My8NPmENHrN5W7OfgZnZJ,https://api.spotify.com/v1/tracks/0My8NPmENHrN...,https://api.spotify.com/v1/audio-analysis/0My8...,232520,4


In [ ]:
df_combined = pd.concat([df_total500, df_total600, df_total800, df_total1500, df_total2000, df_total2500, df_total3000, df_total3500, df_total4000, df_total4477])

In [ ]:
df_combined.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,0.668,0.727,...,0.000019,0.1000,0.939,139.991,audio_features,spotify:track:3pCt2wRdBDa2kCisIdHWgF,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,https://api.spotify.com/v1/audio-analysis/3pCt...,344571,4
1,6Hii26x3qDErVitnGW8QtO,Runtuh,66NmU5epI0ONGmdCRbLpmW,Feby Putri,5boeKAXsTkdFlm6OnkQxQW,Runtuh,2021-10-01,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,0.593,0.287,...,0.000136,0.1190,0.191,127.783,audio_features,spotify:track:6Hii26x3qDErVitnGW8QtO,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,https://api.spotify.com/v1/audio-analysis/6Hii...,223974,4
2,2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,2iDVt6mFbtbDEZG5ax0dTi,Tulus,3R4IAF9ApqYeUQrv1ddyoR,Manusia,2022-03-03,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,0.643,0.436,...,0.088900,0.1150,0.755,71.969,audio_features,spotify:track:2hHeGD57S0BcopfVcmehdl,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,https://api.spotify.com/v1/audio-analysis/2hHe...,242000,4
3,630DpnzdfjdVqv2yLfPbAX,Bertaut,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,0.470,0.131,...,0.002450,0.1090,0.115,118.836,audio_features,spotify:track:630DpnzdfjdVqv2yLfPbAX,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,https://api.spotify.com/v1/audio-analysis/630D...,315961,3
4,4GfK1qOF3uBWidbPlTCQRL,Monokrom,2iDVt6mFbtbDEZG5ax0dTi,Tulus,4szhn3xPmOJklFAcqNvTnQ,Monokrom,2016-08-03,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,0.534,0.462,...,0.000006,0.0974,0.519,88.046,audio_features,spotify:track:4GfK1qOF3uBWidbPlTCQRL,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,https://api.spotify.com/v1/audio-analysis/4GfK...,214567,4


In [ ]:
df_combined.shape

(4383, 25)

In [ ]:
df_combined.to_csv('audio_features_fixed.csv')

# Mengambil lirik

In [ ]:
!pip install syrics

In [ ]:
from syrics.api import Spotify
sp = Spotify("AQDQrMFmKLc7-e3RhqVG-dpok6kMed2O7Qbmofd5F5ep3VLTadYY9pi7K0GIqVIZH81bnTEPNAvw0eyhen-u9NaabdPcmBFgN9HK6GdpuEQXmO2GrKAR_w2h86BdsQwWZiKn_UGatpCZzVHejl13pCuXsQSKG0Bj")

In [ ]:
df_total500.set_index('id', inplace=True)
df_total500.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
3pCt2wRdBDa2kCisIdHWgF,To the Bone,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,0.668,0.727,7,...,0.000019,0.1000,0.939,139.991,audio_features,spotify:track:3pCt2wRdBDa2kCisIdHWgF,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,https://api.spotify.com/v1/audio-analysis/3pCt...,344571,4
6Hii26x3qDErVitnGW8QtO,Runtuh,66NmU5epI0ONGmdCRbLpmW,Feby Putri,5boeKAXsTkdFlm6OnkQxQW,Runtuh,2021-10-01,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,0.593,0.287,3,...,0.000136,0.1190,0.191,127.783,audio_features,spotify:track:6Hii26x3qDErVitnGW8QtO,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,https://api.spotify.com/v1/audio-analysis/6Hii...,223974,4
2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,2iDVt6mFbtbDEZG5ax0dTi,Tulus,3R4IAF9ApqYeUQrv1ddyoR,Manusia,2022-03-03,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,0.643,0.436,6,...,0.088900,0.1150,0.755,71.969,audio_features,spotify:track:2hHeGD57S0BcopfVcmehdl,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,https://api.spotify.com/v1/audio-analysis/2hHe...,242000,4
630DpnzdfjdVqv2yLfPbAX,Bertaut,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,0.470,0.131,6,...,0.002450,0.1090,0.115,118.836,audio_features,spotify:track:630DpnzdfjdVqv2yLfPbAX,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,https://api.spotify.com/v1/audio-analysis/630D...,315961,3
4GfK1qOF3uBWidbPlTCQRL,Monokrom,2iDVt6mFbtbDEZG5ax0dTi,Tulus,4szhn3xPmOJklFAcqNvTnQ,Monokrom,2016-08-03,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,0.534,0.462,6,...,0.000006,0.0974,0.519,88.046,audio_features,spotify:track:4GfK1qOF3uBWidbPlTCQRL,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,https://api.spotify.com/v1/audio-analysis/4GfK...,214567,4


In [ ]:
def process_lyrics_dataframe(df):
    data_lyrics = []
    for index in df.index:
        lyrc = pd.DataFrame()
        track = df.loc[index]['track']
        artist = df.loc[index]['artist']
        data = sp.get_lyrics(index)  # Anda perlu mengganti ini dengan pemanggilan fungsi yang sesuai
        if data is not None:
            data_lines = data['lyrics']['lines']
            data_lyrics_res = []
            for dat in data_lines:
                data_lyrics_res.append(dat['words'])
            data_lyrics_str = ' \n '.join(data_lyrics_res)
            language = data['lyrics']['language']
            lyrc = {'id': index, 'track': track, 'artist': artist, 'lyrics': data_lyrics_str, 'language': language}
            data_lyrics.append(lyrc)

    df_lyrics = pd.DataFrame(data_lyrics)
    return df_lyrics

In [ ]:
processed_df_lyrics = process_lyrics_dataframe(df_total500)

In [ ]:
processed_df_lyrics.head()

,id,track,artist,lyrics,language
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,Pamungkas,Have I ever told you \n I want you to the bone...,en
1,6Hii26x3qDErVitnGW8QtO,Runtuh,Feby Putri,Ku terbangun lagi \n Di antara sepi \n Hanya p...,id
2,2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,Tulus,"Perjalanan membawamu \n Bertemu denganku, ku b...",id
3,630DpnzdfjdVqv2yLfPbAX,Bertaut,Nadin Amizah,"Bun, hidup berjalan seperti bajingan \n ♪ \n S...",id
4,4GfK1qOF3uBWidbPlTCQRL,Monokrom,Tulus,Lembaran foto hitam-putih \n Aku coba ingat la...,id


In [ ]:
processed_df_lyrics.shape

(492, 5)

In [ ]:
df_total600.set_index('id', inplace=True)
df_total600.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
5icOoE6VgqFKohjWWNp0Ac,Here With Me,64KEffDW9EtZ1y2vBYgq8T,Marshmello,6NHS3hV16MZyfcp0nSHdrd,Here With Me,2019-03-08,https://api.spotify.com/v1/tracks/5icOoE6VgqFK...,0.791,0.565,5,...,0.000000,0.156,0.1810,99.961,audio_features,spotify:track:5icOoE6VgqFKohjWWNp0Ac,https://api.spotify.com/v1/tracks/5icOoE6VgqFK...,https://api.spotify.com/v1/audio-analysis/5icO...,156347,4
7zPiwqd1uxBkiHuk2OoBWJ,Cinta Sejati,30qVSJGhPhrZLKG0H9DMA9,Bunga Citra Lestari,3cMuELN6TQ1pF5fkOcdhpv,Cinta Sejati,2012,https://api.spotify.com/v1/tracks/7zPiwqd1uxBk...,0.220,0.402,5,...,0.000194,0.147,0.2240,177.396,audio_features,spotify:track:7zPiwqd1uxBkiHuk2OoBWJ,https://api.spotify.com/v1/tracks/7zPiwqd1uxBk...,https://api.spotify.com/v1/audio-analysis/7zPi...,300933,4
4l0Mvzj72xxOpRrp6h8nHi,Lose You To Love Me,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,3YPFaTR7WMi1Hd4NVKdCJx,Rare,2020-01-10,https://api.spotify.com/v1/tracks/4l0Mvzj72xxO...,0.488,0.343,4,...,0.000000,0.210,0.0978,102.819,audio_features,spotify:track:4l0Mvzj72xxOpRrp6h8nHi,https://api.spotify.com/v1/tracks/4l0Mvzj72xxO...,https://api.spotify.com/v1/audio-analysis/4l0M...,206459,4
3gBFAvpAVSNohs3ESRMYHi,Heaven,05CRzFTp7TouOXPuH6Tapu,Isyana Sarasvati,3GJjPtV0iPKM3fLPvqdURX,Heaven,2018-02-14,https://api.spotify.com/v1/tracks/3gBFAvpAVSNo...,0.540,0.285,3,...,0.000000,0.152,0.4340,87.789,audio_features,spotify:track:3gBFAvpAVSNohs3ESRMYHi,https://api.spotify.com/v1/tracks/3gBFAvpAVSNo...,https://api.spotify.com/v1/audio-analysis/3gBF...,218507,4
5Z3GHaZ6ec9bsiI5BenrbY,Young Dumb & Broke,6LuN9FCkKOj5PcnpouEgny,Khalid,6kf46HbnYCZzP6rjvQHYzg,American Teen,2017-04-27,https://api.spotify.com/v1/tracks/5Z3GHaZ6ec9b...,0.799,0.539,1,...,0.000017,0.165,0.3940,136.948,audio_features,spotify:track:5Z3GHaZ6ec9bsiI5BenrbY,https://api.spotify.com/v1/tracks/5Z3GHaZ6ec9b...,https://api.spotify.com/v1/audio-analysis/5Z3G...,202547,4


In [ ]:
df_lyrics600 = process_lyrics_dataframe(df_total600)

In [ ]:
df_lyrics600.head()

,id,track,artist,lyrics,language
0,5icOoE6VgqFKohjWWNp0Ac,Here With Me,Marshmello,Can I tell you something just between you and ...,en
1,7zPiwqd1uxBkiHuk2OoBWJ,Cinta Sejati,Bunga Citra Lestari,"Manakala hati menggeliat, mengusik renungan \n...",id
2,4l0Mvzj72xxOpRrp6h8nHi,Lose You To Love Me,Selena Gomez,You promised the world and I fell for it \n I ...,en
3,3gBFAvpAVSNohs3ESRMYHi,Heaven,Isyana Sarasvati,Sometimes when I'm with you \n I just get that...,en
4,5Z3GHaZ6ec9bsiI5BenrbY,Young Dumb & Broke,Khalid,So you're still thinking of me \n Just like I ...,en


In [ ]:
df_lyrics600.shape

(98, 5)

In [ ]:
df_total800.set_index('id', inplace=True)
df_total800.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
4EivmOT13NMpNSfTKn9p4s,Like That (feat. Gucci Mane),5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,1MmVkhiwTH0BkNOU3nw5d3,Hot Pink,2019-11-07,https://api.spotify.com/v1/tracks/4EivmOT13NMp...,0.879,0.498,4,...,0.000000,0.0967,0.405,106.044,audio_features,spotify:track:4EivmOT13NMpNSfTKn9p4s,https://api.spotify.com/v1/tracks/4EivmOT13NMp...,https://api.spotify.com/v1/audio-analysis/4Eiv...,163173,4
44ONERBHALxftQNWq6dmyd,Side To Side,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,1QRP5lutJodPixU2EWfnD7,Dangerous Woman,2016-04-01,https://api.spotify.com/v1/tracks/44ONERBHALxf...,0.650,0.736,6,...,0.000000,0.2350,0.612,159.174,audio_features,spotify:track:44ONERBHALxftQNWq6dmyd,https://api.spotify.com/v1/tracks/44ONERBHALxf...,https://api.spotify.com/v1/audio-analysis/44ON...,226160,4
1oPhiw7KmW3JK0gjuSF4YH,Menyimpan Rasa,5nWpcynsaMBFgtGmJODhdi,Devano,0E8TQKFgUP04XaMYK1ls1m,Menyimpan Rasa,2018-03-20,https://api.spotify.com/v1/tracks/1oPhiw7KmW3J...,0.603,0.469,10,...,0.000002,0.1760,0.681,74.998,audio_features,spotify:track:1oPhiw7KmW3JK0gjuSF4YH,https://api.spotify.com/v1/tracks/1oPhiw7KmW3J...,https://api.spotify.com/v1/audio-analysis/1oPh...,210197,4
2YBKZDLTHfoqe7IUEe2z6n,Teman Tapi Menikah,0FYCcpBLjjMMnse9jIRbu3,Dengarkan Dia,61pxeWGW8w4LHpLorcVzL7,Teman Tapi Menikah,2017-02-02,https://api.spotify.com/v1/tracks/2YBKZDLTHfoq...,0.702,0.307,5,...,0.000010,0.0844,0.410,111.987,audio_features,spotify:track:2YBKZDLTHfoqe7IUEe2z6n,https://api.spotify.com/v1/tracks/2YBKZDLTHfoq...,https://api.spotify.com/v1/audio-analysis/2YBK...,222250,4
7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,1uyf3l2d4XYwiEqAb7t7fX,Doo-Wops & Hooligans,2010-10-05,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,0.635,0.841,5,...,0.000000,0.0622,0.424,109.021,audio_features,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,https://api.spotify.com/v1/audio-analysis/7BqB...,220734,4


In [ ]:
df_lyrics800 = process_lyrics_dataframe(df_total800)

In [ ]:
df_lyrics800.head()

,id,track,artist,lyrics,language
0,4EivmOT13NMpNSfTKn9p4s,Like That (feat. Gucci Mane),Doja Cat,"Cállate, Mike Crook \n Burr, burr \n Gucci (Do...",en
1,44ONERBHALxftQNWq6dmyd,Side To Side,Ariana Grande,I've been here all night (Ariana) \n I've been...,en
2,1oPhiw7KmW3JK0gjuSF4YH,Menyimpan Rasa,Devano,Kau ... \n Diam-diam aku jatuh cinta kepadamu ...,id
3,2YBKZDLTHfoqe7IUEe2z6n,Teman Tapi Menikah,Dengarkan Dia,Sadarku akan hadirmu \n Mengubah jalan hidupku...,id
4,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,Bruno Mars,"Oh, her eyes, her eyes \n Make the stars look ...",en


In [ ]:
df_lyrics800.shape

(289, 5)

In [ ]:
df_total1500.set_index('id', inplace=True)
df_total1500.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
7JFFMicliyVArHQgwEqVBS,Alexandra,1DjZI46mVZZZYmmmygRnTw,Reality Club,6xKDnJaJSGOyu2nUmOQlJC,What Do You Really Know?,2019-08-30,https://api.spotify.com/v1/tracks/7JFFMicliyVA...,0.382,0.355,9,...,0.000003,0.2480,0.146,138.419,audio_features,spotify:track:7JFFMicliyVArHQgwEqVBS,https://api.spotify.com/v1/tracks/7JFFMicliyVA...,https://api.spotify.com/v1/audio-analysis/7JFF...,238543,4
7uP1XTPVEI40AZGVKNzITm,Sang Penggoda,6reQVU3OFzQdP0NqZCwrCz,Tata Janeeta,2LtXJXiTmmd2mGbVi7PzLH,Sang Penggoda,2018-04-13,https://api.spotify.com/v1/tracks/7uP1XTPVEI40...,0.354,0.403,0,...,0.000000,0.1240,0.214,118.100,audio_features,spotify:track:7uP1XTPVEI40AZGVKNzITm,https://api.spotify.com/v1/tracks/7uP1XTPVEI40...,https://api.spotify.com/v1/audio-analysis/7uP1...,271804,4
7ju97lgwC2rKQ6wwsf9no9,Rain On Me (with Ariana Grande),1HY2Jd0NmPuamShAr6KMms,Lady Gaga,05c49JgPmL4Uz2ZeqRx5SP,Chromatica,2020-05-29,https://api.spotify.com/v1/tracks/7ju97lgwC2rK...,0.672,0.855,9,...,0.000000,0.3230,0.646,123.056,audio_features,spotify:track:7ju97lgwC2rKQ6wwsf9no9,https://api.spotify.com/v1/tracks/7ju97lgwC2rK...,https://api.spotify.com/v1/audio-analysis/7ju9...,182200,4
1AZjCZSGEjlxUufFQwSszF,Wild Flower (with youjeen),2auC28zjQyVTsiZKNgPRGs,RM,2wGinO7YWLHN2sULIr4a7v,Indigo,2022-12-02,https://api.spotify.com/v1/tracks/1AZjCZSGEjlx...,0.486,0.766,8,...,0.000000,0.1230,0.417,154.992,audio_features,spotify:track:1AZjCZSGEjlxUufFQwSszF,https://api.spotify.com/v1/tracks/1AZjCZSGEjlx...,https://api.spotify.com/v1/audio-analysis/1AZj...,273500,4
20R4HfKloPKgXDqU7UKk3x,Therefore I Am,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,0JGOiO34nwfUdDrD612dOp,Happier Than Ever,2021-07-30,https://api.spotify.com/v1/tracks/20R4HfKloPKg...,0.879,0.365,2,...,0.135000,0.0583,0.663,94.016,audio_features,spotify:track:20R4HfKloPKgXDqU7UKk3x,https://api.spotify.com/v1/tracks/20R4HfKloPKg...,https://api.spotify.com/v1/audio-analysis/20R4...,173539,4


In [ ]:
df_lyrics1500 = process_lyrics_dataframe(df_total1500)

In [ ]:
df_lyrics1500.shape

(475, 5)

In [ ]:
df_total2000.set_index('id', inplace=True)
df_total2000.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
6nxQdXa1uAL0rY72wPZu89,Love Me Now,5y2Xq6xcjJb2jVM54GHK3t,John Legend,7xMjYDrgPLp1ReFGAOyS1O,DARKNESS AND LIGHT,2016-12-02,https://api.spotify.com/v1/tracks/6nxQdXa1uAL0...,0.495,0.762,8,...,0.0000,0.0763,0.708,123.893,audio_features,spotify:track:6nxQdXa1uAL0rY72wPZu89,https://api.spotify.com/v1/tracks/6nxQdXa1uAL0...,https://api.spotify.com/v1/audio-analysis/6nxQ...,210293,4
6NEoeBLQbOMw92qMeLfI40,WHISTLE,41MozSoPIsD1dJM0CLPjZF,BLACKPINK,0FOOodYRlj7gzh7q7IjmNZ,SQUARE ONE,2016-08-08,https://api.spotify.com/v1/tracks/6NEoeBLQbOMw...,0.818,0.723,11,...,0.0000,0.0866,0.607,102.945,audio_features,spotify:track:6NEoeBLQbOMw92qMeLfI40,https://api.spotify.com/v1/tracks/6NEoeBLQbOMw...,https://api.spotify.com/v1/audio-analysis/6NEo...,211632,4
5SjfjoYaRJ5jycgqwV0ow0,Scars,6Ge3YISmUWPCUcSVx8VGor,Keenan Te,5ZFNVSacvOj5XKRkqv39tY,Scars,2023-02-17,https://api.spotify.com/v1/tracks/5SjfjoYaRJ5j...,0.571,0.496,8,...,0.0000,0.1070,0.383,144.925,audio_features,spotify:track:5SjfjoYaRJ5jycgqwV0ow0,https://api.spotify.com/v1/tracks/5SjfjoYaRJ5j...,https://api.spotify.com/v1/audio-analysis/5Sjf...,192000,4
0b9oOr2ZgvyQu88wzixux9,This Is America,73sIBHcqh3Z3NyqHKZ7FOL,Childish Gambino,7arx9qPJexCsDz67El4qvk,This Is America,2018-05-06,https://api.spotify.com/v1/tracks/0b9oOr2ZgvyQ...,0.854,0.463,5,...,0.0000,0.3540,0.549,120.024,audio_features,spotify:track:0b9oOr2ZgvyQu88wzixux9,https://api.spotify.com/v1/tracks/0b9oOr2ZgvyQ...,https://api.spotify.com/v1/audio-analysis/0b9o...,225773,4
7eOkHIcaoUhngxxypM5Ajr,Look At Her Now,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,3YPFaTR7WMi1Hd4NVKdCJx,Rare,2020-01-10,https://api.spotify.com/v1/tracks/7eOkHIcaoUhn...,0.753,0.564,10,...,0.0273,0.3590,0.550,77.342,audio_features,spotify:track:7eOkHIcaoUhngxxypM5Ajr,https://api.spotify.com/v1/tracks/7eOkHIcaoUhn...,https://api.spotify.com/v1/audio-analysis/7eOk...,162596,5


In [ ]:
df_lyrics2000 = process_lyrics_dataframe(df_total2000)

In [ ]:
df_lyrics2000.head()

,id,track,artist,lyrics,language
0,6nxQdXa1uAL0rY72wPZu89,Love Me Now,John Legend,Pulling me further \n Further than I've been b...,en
1,6NEoeBLQbOMw92qMeLfI40,WHISTLE,BLACKPINK,"Hey, boy \n Make 'em whistle like a missile bo...",ko
2,5SjfjoYaRJ5jycgqwV0ow0,Scars,Keenan Te,We're whispering in circles again \n We're usi...,en
3,0b9oOr2ZgvyQu88wzixux9,This Is America,Childish Gambino,"Yeah, yeah, yeah, yeah, yeah \n Yeah, yeah, ye...",en
4,7eOkHIcaoUhngxxypM5Ajr,Look At Her Now,Selena Gomez,Uhh \n Yeah \n They fell in love one summer \n...,en


In [ ]:
df_total2500.set_index('id', inplace=True)
df_total2500.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
55kuHkT2CWhWKF6CoRDnJq,Dumhdurum,2uWcrwgWmZcQc3IPBs3tfU,Apink,49NKBZDb92AQlMFypkZRTF,LOOK,2020-04-13,https://api.spotify.com/v1/tracks/55kuHkT2CWhW...,0.653,0.923,8,...,0.0,0.137,0.6920,114.902,audio_features,spotify:track:55kuHkT2CWhWKF6CoRDnJq,https://api.spotify.com/v1/tracks/55kuHkT2CWhW...,https://api.spotify.com/v1/audio-analysis/55ku...,208053,4
6FtLXwyMr1t1C7xTUBL9uY,MOVE,5KOTrsUUc0VUVH3mR5StNC,SIXC (6 crazy),0dK8TElRE9K5AyO31YptuE,PRODUCE X 101 - 31 Boys 5 Concepts,2019-07-05,https://api.spotify.com/v1/tracks/6FtLXwyMr1t1...,0.733,0.782,8,...,0.0,0.403,0.8180,106.892,audio_features,spotify:track:6FtLXwyMr1t1C7xTUBL9uY,https://api.spotify.com/v1/tracks/6FtLXwyMr1t1...,https://api.spotify.com/v1/audio-analysis/6FtL...,185400,4
14pjnaIqkpReO5D0tHofAS,Daisy,1wKpMkucynaTfG8lyPprYV,PENTAGON,1ASYbBYBwV6Rcfc2ycqmlK,WE:TH,2020-10-12,https://api.spotify.com/v1/tracks/14pjnaIqkpRe...,0.563,0.869,5,...,0.0,0.138,0.4800,85.016,audio_features,spotify:track:14pjnaIqkpReO5D0tHofAS,https://api.spotify.com/v1/tracks/14pjnaIqkpRe...,https://api.spotify.com/v1/audio-analysis/14pj...,189000,4
2MdoXFLKNUnTdgWZisagEp,You Are My Everything (feat. Red),4rUYk0fV0Z4pOtwVbEAyK9,Glenn Fredly,5qPDSS9JkOf6As1DqdhKpF,OST. Cinta Silver,2005-06-24,https://api.spotify.com/v1/tracks/2MdoXFLKNUnT...,0.590,0.689,4,...,0.0,0.190,0.5460,104.985,audio_features,spotify:track:2MdoXFLKNUnTdgWZisagEp,https://api.spotify.com/v1/tracks/2MdoXFLKNUnT...,https://api.spotify.com/v1/audio-analysis/2Mdo...,306227,4
7KokYm8cMIXCsGVmUvKtqf,Karma,06HL4z0CvFAxyc27GXpf02,Taylor Swift,151w1FgRZfnKZA9FEcg9Z3,Midnights,2022-10-21,https://api.spotify.com/v1/tracks/7KokYm8cMIXC...,0.642,0.620,8,...,0.0,0.483,0.0969,90.020,audio_features,spotify:track:7KokYm8cMIXCsGVmUvKtqf,https://api.spotify.com/v1/tracks/7KokYm8cMIXC...,https://api.spotify.com/v1/audio-analysis/7Kok...,204852,4


In [ ]:
df_lyrics2500 = process_lyrics_dataframe(df_total2500)

In [ ]:
df_lyrics2500.head()

,id,track,artist,lyrics,language
0,55kuHkT2CWhWKF6CoRDnJq,Dumhdurum,Apink,Hey 느낌이 너무 이상한 걸 좀 \n 나를 위해 보내준다는 말이 \n 직감은 두려...,ko
1,14pjnaIqkpReO5D0tHofAS,Daisy,PENTAGON,거짓말 같은 밤 거지 같은 기억 \n 이젠 다시 돌아가고 싶지 않아 \n \n 사...,ko
2,2MdoXFLKNUnTdgWZisagEp,You Are My Everything (feat. Red),Glenn Fredly,Cruising when the sun goes down \n Across the ...,en
3,7KokYm8cMIXCsGVmUvKtqf,Karma,Taylor Swift,You're talking shit for the hell of it \n Addi...,en
4,2cYqizR4lgvp4Qu6IQ3qGN,BUTTERFLY EFFECT,Travis Scott,All the commas \n Murda on the beat so it's no...,en


In [ ]:
df_lyrics2500.shape

(475, 5)

In [ ]:
df_total3000.set_index('id', inplace=True)
df_total3000.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
29euE2RV1DoN6aJXm3wmcS,Jadi Milikku,0JvUFDnuWPbuyRa6ptVkLk,Marcell,4lbLWbnI1sbBsPK5lYNprT,Jadi Milikku,2017-07-06,https://api.spotify.com/v1/tracks/29euE2RV1DoN...,0.586,0.404,7,...,0.000000,0.1010,0.391,115.913,audio_features,spotify:track:29euE2RV1DoN6aJXm3wmcS,https://api.spotify.com/v1/tracks/29euE2RV1DoN...,https://api.spotify.com/v1/audio-analysis/29eu...,251897,4
3D110t9ctcoOoylXsc3iR8,Thick And Thin,49tQo2QULno7gxHutgccqF,LANY,6SWmGozzQDUaczHXMuE8Za,Malibu Nights,2018-10-05,https://api.spotify.com/v1/tracks/3D110t9ctcoO...,0.697,0.551,5,...,0.000000,0.0762,0.339,99.996,audio_features,spotify:track:3D110t9ctcoOoylXsc3iR8,https://api.spotify.com/v1/tracks/3D110t9ctcoO...,https://api.spotify.com/v1/audio-analysis/3D11...,212002,4
1WjGvMbHE23jEipydV80Mu,FAKE LOVE - Rocking Vibe Mix,3Nrfpe0tUJi4K4DXYWgMUX,BTS,43wFM1HquliY3iwKWzPN4y,Love Yourself 結 'Answer',2018-08-24,https://api.spotify.com/v1/tracks/1WjGvMbHE23j...,0.466,0.804,2,...,0.000000,0.4320,0.352,154.901,audio_features,spotify:track:1WjGvMbHE23jEipydV80Mu,https://api.spotify.com/v1/tracks/1WjGvMbHE23j...,https://api.spotify.com/v1/audio-analysis/1WjG...,238494,4
7kt9e9LFSpN1zQtYEl19o1,this is me trying,06HL4z0CvFAxyc27GXpf02,Taylor Swift,2fenSS68JI1h4Fo296JfGr,folklore,2020-07-24,https://api.spotify.com/v1/tracks/7kt9e9LFSpN1...,0.515,0.476,9,...,0.000047,0.1710,0.412,135.886,audio_features,spotify:track:7kt9e9LFSpN1zQtYEl19o1,https://api.spotify.com/v1/tracks/7kt9e9LFSpN1...,https://api.spotify.com/v1/audio-analysis/7kt9...,195097,4
5oO3drDxtziYU2H1X23ZIp,Love On The Brain,5pKCCKE2ajJHZ9KAiaK11H,Rihanna,4UlGauD7ROb3YbVOFMgW5u,ANTI (Deluxe),2016-01-28,https://api.spotify.com/v1/tracks/5oO3drDxtziY...,0.509,0.637,4,...,0.000011,0.0789,0.378,172.006,audio_features,spotify:track:5oO3drDxtziYU2H1X23ZIp,https://api.spotify.com/v1/tracks/5oO3drDxtziY...,https://api.spotify.com/v1/audio-analysis/5oO3...,224000,3


In [ ]:
df_lyrics3000 = process_lyrics_dataframe(df_total3000)

In [ ]:
df_lyrics3000.head()

,id,track,artist,lyrics,language
0,29euE2RV1DoN6aJXm3wmcS,Jadi Milikku,Marcell,Ku rasakan cinta \n Sejak pertama kujumpa diri...,id
1,3D110t9ctcoOoylXsc3iR8,Thick And Thin,LANY,Late night \n Your eyes fell to the floor \n I...,en
2,1WjGvMbHE23jEipydV80Mu,FAKE LOVE - Rocking Vibe Mix,BTS,널 위해서라면 난 \n 슬퍼도 기쁜 척 할 수가 있었어 \n 널 위해서라면 난 \n...,ko
3,7kt9e9LFSpN1zQtYEl19o1,this is me trying,Taylor Swift,I've been having a hard time adjusting \n I ha...,en
4,5oO3drDxtziYU2H1X23ZIp,Love On The Brain,Rihanna,And you got me like oh \n What you want from m...,en


In [ ]:
df_lyrics3000.shape

(478, 5)

In [ ]:
df_total3500.set_index('id', inplace=True)
df_total3500.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
1y3bE5i57eUm3hfLAc7h08,Walking The Wire,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,33pt9HBdGlAbRGBHQgsZsU,Evolve,2017-06-23,https://api.spotify.com/v1/tracks/1y3bE5i57eUm...,0.540,0.761,4,...,0.000000,0.0941,0.147,100.008,audio_features,spotify:track:1y3bE5i57eUm3hfLAc7h08,https://api.spotify.com/v1/tracks/1y3bE5i57eUm...,https://api.spotify.com/v1/audio-analysis/1y3b...,232907,4
0heY4hxgRs6s1wrMtrS3YJ,Tak Pernah Padam,4sZCNSc9bpEa6bjM8Z3NOu,Sandhy Sondoro,3kOVbZ8Ccl83UB9cGTvvi5,Find The Way,2011-12-16,https://api.spotify.com/v1/tracks/0heY4hxgRs6s...,0.667,0.456,8,...,0.000000,0.1700,0.551,74.987,audio_features,spotify:track:0heY4hxgRs6s1wrMtrS3YJ,https://api.spotify.com/v1/tracks/0heY4hxgRs6s...,https://api.spotify.com/v1/audio-analysis/0heY...,241107,4
3p287E4Y1LOqbSICfbv1hk,The Days We Loved,3l0MJ9LCiBlMyOMaz485Q5,Baek Ji Young,3giN7KiHhfRJb2L8LGToHu,"The World of the Married, Pt. 6 (Original Tele...",2020-05-01,https://api.spotify.com/v1/tracks/3p287E4Y1LOq...,0.367,0.627,10,...,0.000000,0.1050,0.380,137.013,audio_features,spotify:track:3p287E4Y1LOqbSICfbv1hk,https://api.spotify.com/v1/tracks/3p287E4Y1LOq...,https://api.spotify.com/v1/audio-analysis/3p28...,241053,4
38iufYNK4hUzS649Oe2kZy,Burn It (feat. MAX),5RmQ8k4l3HZ8JoPb4mNsML,Agust D,1Pp8t7yn2E3rz3R7ZqPn1O,D-2,2020-05-22,https://api.spotify.com/v1/tracks/38iufYNK4hUz...,0.658,0.776,6,...,0.000002,0.2720,0.336,114.044,audio_features,spotify:track:38iufYNK4hUzS649Oe2kZy,https://api.spotify.com/v1/tracks/38iufYNK4hUz...,https://api.spotify.com/v1/audio-analysis/38iu...,192107,4
2aNkofLzdkgsCpC0AzEJB4,What We Remember,3ahVFszO9ucf8JQLCNQeYA,Anggun,3m7KxVv2s1VhrfJh6cinck,8,2017-12-08,https://api.spotify.com/v1/tracks/2aNkofLzdkgs...,0.800,0.677,2,...,0.000428,0.1760,0.387,114.041,audio_features,spotify:track:2aNkofLzdkgsCpC0AzEJB4,https://api.spotify.com/v1/tracks/2aNkofLzdkgs...,https://api.spotify.com/v1/audio-analysis/2aNk...,166294,4


In [ ]:
df_lyrics3500 = process_lyrics_dataframe(df_total3500)

In [ ]:
df_lyrics3500.head()

,id,track,artist,lyrics,language
0,1y3bE5i57eUm3hfLAc7h08,Walking The Wire,Imagine Dragons,Do you feel the same when I'm away from you? (...,en
1,0heY4hxgRs6s1wrMtrS3YJ,Tak Pernah Padam,Sandhy Sondoro,Senyumanmu \n Masih jelas terkenang \n Hadir s...,id
2,3p287E4Y1LOqbSICfbv1hk,The Days We Loved,Baek Ji Young,희미해진 기억 \n 어느 봄날의 꿈들 \n 전부 사라지고 시린 밤은 깊어가고 \n ...,ko
3,38iufYNK4hUzS649Oe2kZy,Burn It (feat. MAX),Agust D,(Yeah) I see the ashes falling out your window...,ko
4,2aNkofLzdkgsCpC0AzEJB4,What We Remember,Anggun,Planes fly over my head \n Drawing lines in th...,en


In [ ]:
df_lyrics3500.shape

(475, 5)

In [ ]:
df_total4000.set_index('id', inplace=True)
df_total4000.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
61HVbcNeRACZpyvHrc3AnD,All Is Found,3MyB0MyKec3Jq0AH2llk7v,Evan Rachel Wood,4M07HWIlZr7zoXoxDHR5mz,Frozen 2 (Original Motion Picture Soundtrack/D...,2019-11-15,https://api.spotify.com/v1/tracks/61HVbcNeRACZ...,0.228,0.204,5,...,0.000011,0.092,0.195,83.372,audio_features,spotify:track:61HVbcNeRACZpyvHrc3AnD,https://api.spotify.com/v1/tracks/61HVbcNeRACZ...,https://api.spotify.com/v1/audio-analysis/61HV...,125439,4
6smhpAoDKenJyDztNHf0HH,Thinking About You,2XnBwblw31dfGnspMIwgWz,Axwell /\ Ingrosso,6Ei5WWek37m1x9AjpSXTcJ,More Than You Know,2017-07-28,https://api.spotify.com/v1/tracks/6smhpAoDKenJ...,0.678,0.842,8,...,0.000000,0.202,0.493,122.016,audio_features,spotify:track:6smhpAoDKenJyDztNHf0HH,https://api.spotify.com/v1/tracks/6smhpAoDKenJ...,https://api.spotify.com/v1/audio-analysis/6smh...,204587,4
28foEMQSiO6arnHRX8HdHj,Like always,6C4irZ60X8G7UimMRQiCcg,X1,3zgalFg8VBaUSHKeLBVGun,QUANTUM LEAP,2019-08-27,https://api.spotify.com/v1/tracks/28foEMQSiO6a...,0.782,0.879,5,...,0.000000,0.047,0.822,119.978,audio_features,spotify:track:28foEMQSiO6arnHRX8HdHj,https://api.spotify.com/v1/tracks/28foEMQSiO6a...,https://api.spotify.com/v1/audio-analysis/28fo...,204893,4
7gzdvN8XbAXwYFOOUj7CxM,hormat kepada angin,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,2H7YgnGyrAi2AZsskG6Ows,kalah bertaruh,2021-05-26,https://api.spotify.com/v1/tracks/7gzdvN8XbAXw...,0.480,0.354,5,...,0.031600,0.137,0.221,138.071,audio_features,spotify:track:7gzdvN8XbAXwYFOOUj7CxM,https://api.spotify.com/v1/tracks/7gzdvN8XbAXw...,https://api.spotify.com/v1/audio-analysis/7gzd...,355627,4
4mU5iXHeLgbR94siF7p1sY,Six Feet Under,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,2ODvWsOgouMbaA5xf0RkJe,Starboy,2016-11-25,https://api.spotify.com/v1/tracks/4mU5iXHeLgbR...,0.772,0.504,6,...,0.000000,0.125,0.244,140.035,audio_features,spotify:track:4mU5iXHeLgbR94siF7p1sY,https://api.spotify.com/v1/tracks/4mU5iXHeLgbR...,https://api.spotify.com/v1/audio-analysis/4mU5...,237573,4


In [ ]:
df_lyrics4000 = process_lyrics_dataframe(df_total4000)

In [ ]:
df_lyrics4000.shape

(465, 5)

In [ ]:
df_lyrics4000.head()

,id,track,artist,lyrics,language
0,61HVbcNeRACZpyvHrc3AnD,All Is Found,Evan Rachel Wood,Where the North wind meets the sea \n There's ...,en
1,6smhpAoDKenJyDztNHf0HH,Thinking About You,Axwell /\ Ingrosso,"Think about love, think about hate \n Think ab...",en
2,28foEMQSiO6arnHRX8HdHj,Like always,X1,바람이 불어 너를 스치면 \n 아팠던 마음도 가져갔으면 \n 무슨 일 있어 보여 어...,ko
3,7gzdvN8XbAXwYFOOUj7CxM,hormat kepada angin,Nadin Amizah,Hormat kepada angin \n Datang membawa musim \n...,ms
4,4mU5iXHeLgbR94siF7p1sY,Six Feet Under,The Weeknd,Ask around about her \n She don't get emotiona...,en


In [ ]:
df_total4477.set_index('id', inplace=True)
df_total4477.head()

,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
id,,,,,,,,,,,,,,,,,,,,,
2BcwzjPvVSkek2ZLdh7C9J,Sing Me to Sleep - Marshmello Remix,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,4wQNBX0Mp50Q2JFSEDW5G1,Sing Me to Sleep,2016-06-03,https://api.spotify.com/v1/tracks/2BcwzjPvVSke...,0.620,0.893,4,...,0.000304,0.1590,0.160,101.089,audio_features,spotify:track:2BcwzjPvVSkek2ZLdh7C9J,https://api.spotify.com/v1/tracks/2BcwzjPvVSke...,https://api.spotify.com/v1/audio-analysis/2Bcw...,192480,4
3w9mof4ZxG3z9huYwHFZ1i,That's It (feat. Gucci Mane & 2 Chainz),64M6ah0SkkRsnPGtGiRAbb,Bebe Rexha,6t5D6LEgHxqUVOxJItkzfb,All Your Fault: Pt. 2,2017-08-11,https://api.spotify.com/v1/tracks/3w9mof4ZxG3z...,0.783,0.361,0,...,0.007620,0.0861,0.413,75.000,audio_features,spotify:track:3w9mof4ZxG3z9huYwHFZ1i,https://api.spotify.com/v1/tracks/3w9mof4ZxG3z...,https://api.spotify.com/v1/audio-analysis/3w9m...,207090,4
6FT9FZccFD6nE8dMNslz2n,Walk On Water,0RqtSIYZmd4fiBKVFqyIqD,Thirty Seconds To Mars,0XcHdI2ZyNADjfvo5Ubs39,AMERICA,2018-04-06,https://api.spotify.com/v1/tracks/6FT9FZccFD6n...,0.495,0.787,8,...,0.000000,0.1670,0.388,140.003,audio_features,spotify:track:6FT9FZccFD6nE8dMNslz2n,https://api.spotify.com/v1/tracks/6FT9FZccFD6n...,https://api.spotify.com/v1/audio-analysis/6FT9...,185467,4
46coHhbK0YFKlnh3qYtVBB,I Did It,2fS9sWFJcWN8wVhYbCfdC7,Teza Sumendra,0ULXt1LoNwUlyWVbAjfQMU,I Did It,2018-03-16,https://api.spotify.com/v1/tracks/46coHhbK0YFK...,0.726,0.681,5,...,0.000047,0.1470,0.635,116.954,audio_features,spotify:track:46coHhbK0YFKlnh3qYtVBB,https://api.spotify.com/v1/tracks/46coHhbK0YFK...,https://api.spotify.com/v1/audio-analysis/46co...,336466,4
0My8NPmENHrN5W7OfgZnZJ,The One,4BxCuXFJrSWGi1KHcVqaU4,Kodaline,3IJ5RDbQeiWIf7Fo601JwE,Coming Up for Air (Expanded Edition),2015-02-09,https://api.spotify.com/v1/tracks/0My8NPmENHrN...,0.430,0.587,9,...,0.000088,0.1580,0.332,156.006,audio_features,spotify:track:0My8NPmENHrN5W7OfgZnZJ,https://api.spotify.com/v1/tracks/0My8NPmENHrN...,https://api.spotify.com/v1/audio-analysis/0My8...,232520,4


In [ ]:
df_lyrics4477 = process_lyrics_dataframe(df_total4477)

In [ ]:
df_lyrics4477.head()

,id,track,artist,lyrics,language
0,2BcwzjPvVSkek2ZLdh7C9J,Sing Me to Sleep - Marshmello Remix,Alan Walker,"Wait a second, let me catch my breath \n Remin...",en
1,3w9mof4ZxG3z9huYwHFZ1i,That's It (feat. Gucci Mane & 2 Chainz),Bebe Rexha,"Huh, it's Bebe \n Gucci \n 2 Chainz, yah \n Mu...",en
2,6FT9FZccFD6nE8dMNslz2n,Walk On Water,Thirty Seconds To Mars,"Oh oh, oh oh \n Oh oh, oh oh \n Oh oh, oh oh (...",en
3,0My8NPmENHrN5W7OfgZnZJ,The One,Kodaline,"Tell me, tell me that you want me \n And I'll ...",en
4,08Wc3X7XNSyF8WYBPt2L8E,"Holiday (feat. Snoop Dogg, John Legend & Takeoff)",Calvin Harris,"Look, open your eyes, déjà vu \n In ways I kne...",en


In [ ]:
df_lyrics4477.shape

(448, 5)

In [ ]:
df_lyrics_combined = pd.concat([processed_df_lyrics, df_lyrics600, df_lyrics800, df_lyrics1500, df_lyrics2000, df_lyrics2500, df_lyrics3000, df_lyrics3500, df_lyrics4000, df_lyrics4477])

In [ ]:
df_lyrics_combined.head()

,id,track,artist,lyrics,language
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,Pamungkas,Have I ever told you \n I want you to the bone...,en
1,6Hii26x3qDErVitnGW8QtO,Runtuh,Feby Putri,Ku terbangun lagi \n Di antara sepi \n Hanya p...,id
2,2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,Tulus,"Perjalanan membawamu \n Bertemu denganku, ku b...",id
3,630DpnzdfjdVqv2yLfPbAX,Bertaut,Nadin Amizah,"Bun, hidup berjalan seperti bajingan \n ♪ \n S...",id
4,4GfK1qOF3uBWidbPlTCQRL,Monokrom,Tulus,Lembaran foto hitam-putih \n Aku coba ingat la...,id


In [ ]:
df_lyrics_combined.shape

(4157, 5)

In [ ]:
df_lyrics_combined.to_csv('lyrics_total.csv')